# Python programming for SAS programmer

Python is one of the most popualr program nowadays. This program will show how SAS programmer will use Pandas to conduct SAS-like task. 

## Pandas library
Python has Pandas library that is very similar to data procedure that SAS programmmer can use.  
Data Structure comparison between Pandas and SAS dataset


<table a="left>
<tr> <th>Pandas   </th> <th>      SAS     </th> </tr>
<tr> <td>DataFrame  </td> <td>    data set  </td> </tr>
<tr> <td>column   </td> <td>    variable  </td> </tr>
<tr> <td>row   </td> <td>    observation  </td> </tr>
<tr> <td>groupby   </td> <td>    BY-group  </td> </tr>   
<tr> <td>NaN   </td> <td>    .  </td> </tr>
</table>

## Xport library
Xport library can read SAS xport files from the local drive and write to the local drive.

## sas7bdat library
sas7bdata libray can read SAS sas7bdat files from the local drive. 

In [1]:
### Import modules
from pandas import DataFrame
import pandas as pd
import numpy as np
import xport  ## write sas datasets to local drive
from sas7bdat import SAS7BDAT  ## read SAS datasets from local drive

## Read data from the local drive
Programmer can use 

In [2]:
### Read SAS xport datasets
with open('./data/in/demog.xpt', 'rb') as f: 
    df_dm = xport.to_columns(f)
    df_dm2 = pd.DataFrame(df_dm) 
print('Read SAS xport demog data')
print(df_dm2.head())

### Read exposure datasets
with open('./data/in/expo.xpt', 'rb') as f: 
    df_ex = xport.to_columns(f)
    df_ex2 = pd.DataFrame(df_ex) 
    
### Read SAS sas7bdat datasets
with SAS7BDAT('./data/in/adtteos.sas7bdat') as file: 
    df_dm3 = file.to_data_frame()
print(df_dm3.head())

### Read excel files 
df_ts = pd.io.excel.read_excel('./data/in/ts.xlsx', sheet_name=0)
print("Read excel files from the local drive")
print(df_ts.head())

Read SAS xport demog data
    AGE   AGEU                                              RACEC    SEXC  \
0  63.0  YEARS  HISPANIC (MEXICAN - AMERICAN, MEXICO, CENTRAL ...  Female   
1  64.0  YEARS  HISPANIC (MEXICAN - AMERICAN, MEXICO, CENTRAL ...    Male   
2  71.0  YEARS                                          CAUCASIAN    Male   
3  74.0  YEARS                                          CAUCASIAN    Male   
4  77.0  YEARS                                          CAUCASIAN  Female   

  SITEID       STUDYID SUBJID  
0    701  CDISCPILOT01   1015  
1    701  CDISCPILOT01   1023  
2    701  CDISCPILOT01   1028  
3    701  CDISCPILOT01   1033  
4    701  CDISCPILOT01   1034  
    SUBJID SITEID FASFL SAFFL        TRTP  TRTPN          PARAM PARAMCD  \
0  310-001    310     Y     Y     Control    1.0  Days to Death   DEATH   
1  310-002    310     Y     Y     Control    1.0  Days to Death   DEATH   
2  310-003    310     Y     Y  Study Drug    2.0  Days to Death   DEATH   
3  310-004    310  

## Information of dataset like proc contents

dataframe info() function can describe the basic information of dataset

<table>
<tr> <th> Columns in DataFrame  </th> <th> variables in SAS data set </th> </tr>
<tr> <td>float or int   </td> <td>    num  </td> </tr>
<tr> <td>object   </td> <td>    char  </td> </tr>
</table>

datarame describe() is not proc summary.  It provides simple summary descriptics of numeric variables.

In [3]:
print(df_dm2.info())
print(df_dm2.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Data columns (total 7 columns):
AGE        254 non-null float64
AGEU       254 non-null object
RACEC      254 non-null object
SEXC       254 non-null object
SITEID     254 non-null object
STUDYID    254 non-null object
SUBJID     254 non-null object
dtypes: float64(1), object(6)
memory usage: 14.0+ KB
None
              AGE
count  254.000000
mean    75.086614
std      8.246234
min     51.000000
25%     70.000000
50%     77.000000
75%     81.000000
max     89.000000


## Data Manipulation


data df_dm2;   <br>
&nbsp;&nbsp;&nbsp;&nbsp;    set df_dm2:  
&nbsp;&nbsp;&nbsp;&nbsp;     if SEXC = 'Male' then SEX = 'M';
&nbsp;&nbsp;     if SEXC = 'Female' then SEX = 'F';  
&nbsp;&nbsp;&nbsp;&nbsp;    DOMAIN = 'DM';
&nbsp;&nbsp;     COUNTRY = 'USA';  
&nbsp;&nbsp;&nbsp;&nbsp;    SUBJID = STUDYID || '-' || SITEID || '-' || SUBJID;  
run;

Above SAS programs can be written in python dataframe. 

In [4]:
### Prepare variables 
df_dm2['SEX'] = df_dm2.SEXC.replace(['Male','Female'], ['M','F']) # Create SEX variable
df_dm2['DOMAIN'], df_dm2['COUNTRY'] = 'DM', 'USA' # Create varialbes DOMAIN & COUNTRY
df_dm2['USUBJID'] = df_dm2.STUDYID + '-' + df_dm2.SITEID + '-' + df_dm2.SUBJID  # create USUBJID

for index, _df1 in df_dm2.iterrows():
    ## Create Ethnic
    if _df1['RACEC'].__contains__('HISPANIC'):
        df_dm2.loc[index, 'ETHNIC'] = 'HISPANIC OR LATINO'
    else:
        df_dm2.loc[index, 'ETHNIC'] = 'NOT HISPANIC OR LATINO'
        
    ## Create Race    
    if _df1['RACEC'].__contains__('HISPANIC'):
        df_dm2.loc[index, 'RACE'] = 'WHITE'
    elif _df1['RACEC'].__contains__('CAUCASIAN'):
        df_dm2.loc[index, 'RACE'] = 'WHITE'
    elif _df1['RACEC'].__contains__('AFRICAN'):
        df_dm2.loc[index, 'RACE'] = 'BLACK'
    elif _df1['RACEC'].__contains__('OTHER'):
        df_dm2.loc[index, 'RACE'] = 'OTHER'
df_dm2.head()

,AGE,AGEU,RACEC,SEXC,SITEID,STUDYID,SUBJID,SEX,DOMAIN,COUNTRY,USUBJID,ETHNIC,RACE
0,63.0,YEARS,"HISPANIC (MEXICAN - AMERICAN, MEXICO, CENTRAL ...",Female,701,CDISCPILOT01,1015,F,DM,USA,CDISCPILOT01-701-1015,HISPANIC OR LATINO,WHITE
1,64.0,YEARS,"HISPANIC (MEXICAN - AMERICAN, MEXICO, CENTRAL ...",Male,701,CDISCPILOT01,1023,M,DM,USA,CDISCPILOT01-701-1023,HISPANIC OR LATINO,WHITE
2,71.0,YEARS,CAUCASIAN,Male,701,CDISCPILOT01,1028,M,DM,USA,CDISCPILOT01-701-1028,NOT HISPANIC OR LATINO,WHITE
3,74.0,YEARS,CAUCASIAN,Male,701,CDISCPILOT01,1033,M,DM,USA,CDISCPILOT01-701-1033,NOT HISPANIC OR LATINO,WHITE
4,77.0,YEARS,CAUCASIAN,Female,701,CDISCPILOT01,1034,F,DM,USA,CDISCPILOT01-701-1034,NOT HISPANIC OR LATINO,WHITE


## Concatenate variables

data df_ex2;   <br>
&nbsp;&nbsp;&nbsp;&nbsp;    set df_ex2:  
&nbsp;&nbsp;&nbsp;&nbsp;    EXSTDTC = EXSTDD || EXSTMM || EXSTYY;   
&nbsp;&nbsp;&nbsp;&nbsp;    EXENDTC = EXENDD || EXENMM || EXENYY;  
run;


In [5]:
### Prepare exposure variables
df_ex2['EXSTDTC'] = df_ex2['EXSTDD'] + '-' + df_ex2['EXSTMM'] + '-' + df_ex2['EXSTYY']  # exposure start date
df_ex2['EXENDTC'] = df_ex2['EXENDD'] + '-' + df_ex2['EXENMM'] + '-' + df_ex2['EXENYY']  # exposure end date

## Sort data and first.SUBJID and last.SUBJID

proc sort data=df_ex3 out=df_ex4;
&nbsp;&nbsp;&nbsp;&nbsp; by subjid exstdtc;
run;

data df_ex4_f (keep=subjid exstdtc rename=(exstdtc=rfstdtc));   <br>
&nbsp;&nbsp;&nbsp;&nbsp;    set df_ex4:  
&nbsp;&nbsp;&nbsp;&nbsp;    if first.subjid;  
run;

data df_ex4_l (keep=subjid exendtc rename=(exendtc=rfendtc));   <br>
&nbsp;&nbsp;&nbsp;&nbsp;    set df_ex4:  
&nbsp;&nbsp;&nbsp;&nbsp;    if last.subjid;  
run;

In [6]:
### Fine the first and last exposure date
df_ex4 = df_ex2.sort_values(by =['SUBJID','EXSTDTC'], ascending=[True, True] )  # sort by subjid and expo date
df_ex4_f = df_ex4.groupby('SUBJID').first()  # pick the first exposure date of the subject
df_ex4_f2 = df_ex4_f.reset_index()  # reset index so that SUBJID is column
df_ex4_f3 = df_ex4_f2[['SUBJID','EXSTDTC']].rename(columns={'EXSTDTC':'RFSTDTC'})  # select variables
df_ex4_l = df_ex4.groupby('SUBJID').last()  # pick the last exposure date of the subject
df_ex4_l2 = df_ex4_l.reset_index()  # reset index so that SUBJID is column
df_ex4_l3 = df_ex4_l2[['SUBJID','EXENDTC']].rename(columns={'EXENDTC':'RFENDTC'})  # select variables

### Merge SAS datasets

data df_ex5;   <br>
&nbsp;&nbsp;&nbsp;&nbsp;    merge df_ex4_f3 df_ex_l3:  
&nbsp;&nbsp;&nbsp;&nbsp;    by subjid;  
run;

data df_dm3;   <br>
&nbsp;&nbsp;&nbsp;&nbsp;    merge df_dm2(in=a) df_ex5:  
&nbsp;&nbsp;&nbsp;&nbsp;    by subjid; 
&nbsp;&nbsp;&nbsp;&nbsp;    if a;   
run;

In [7]:
### merge first and last exposure date
df_ex5 = pd.merge(df_ex4_f3, df_ex4_l3, on='SUBJID')

### merge exposure data to demog data
df_dm3 = pd.merge(df_dm2, df_ex5, on='SUBJID', how='left')  # merge exposure information to dm

df_dm3.head()

,AGE,AGEU,RACEC,SEXC,SITEID,STUDYID,SUBJID,SEX,DOMAIN,COUNTRY,USUBJID,ETHNIC,RACE,RFSTDTC,RFENDTC
0,63.0,YEARS,"HISPANIC (MEXICAN - AMERICAN, MEXICO, CENTRAL ...",Female,701,CDISCPILOT01,1015,F,DM,USA,CDISCPILOT01-701-1015,HISPANIC OR LATINO,WHITE,2014-01-02,2014-07-02
1,64.0,YEARS,"HISPANIC (MEXICAN - AMERICAN, MEXICO, CENTRAL ...",Male,701,CDISCPILOT01,1023,M,DM,USA,CDISCPILOT01-701-1023,HISPANIC OR LATINO,WHITE,2012-08-05,2012-09-01
2,71.0,YEARS,CAUCASIAN,Male,701,CDISCPILOT01,1028,M,DM,USA,CDISCPILOT01-701-1028,NOT HISPANIC OR LATINO,WHITE,2013-07-19,2014-01-14
3,74.0,YEARS,CAUCASIAN,Male,701,CDISCPILOT01,1033,M,DM,USA,CDISCPILOT01-701-1033,NOT HISPANIC OR LATINO,WHITE,2014-03-18,2014-03-31
4,77.0,YEARS,CAUCASIAN,Female,701,CDISCPILOT01,1034,F,DM,USA,CDISCPILOT01-701-1034,NOT HISPANIC OR LATINO,WHITE,2014-07-01,2014-12-30


## Frequency of data

proc freq data=df_dm3;  
&nbsp;&nbsp;&nbsp;&nbsp;    table sex:  
run;

In [8]:
print(df_dm3['SEX'].value_counts())

F    143
M    111
Name: SEX, dtype: int64


## subsetting data

data df_dm3_male;  
&nbsp;&nbsp;&nbsp;&nbsp;  set df_dm3:  
&nbsp;&nbsp;&nbsp;&nbsp;  if sex = 'M';  
run;

In [9]:
df_dm3_male = df_dm3[df_dm3.SEX == 'M']
print(df_dm3_male['SEX'].value_counts())

M    111
Name: SEX, dtype: int64


## If Statement

data df_dm3_agegroup;  
&nbsp;&nbsp;&nbsp;&nbsp;  set df_dm3:  
&nbsp;&nbsp;&nbsp;&nbsp;  if age > 60 then agegroup = 'age group 1';  
&nbsp;&nbsp;&nbsp;&nbsp;  else agegroup = 'age group 2';  
run;

In [10]:
df_dm3_agegroup = df_dm3.copy()
df_dm3_agegroup['AGEGROUP'] = np.where(df_dm3.AGE > 60, 'age group 1', 'age group 2')
df_dm3_agegroup[['AGE', 'AGEGROUP']].head()

,AGE,AGEGROUP
0,63.0,age group 1
1,64.0,age group 1
2,71.0,age group 1
3,74.0,age group 1
4,77.0,age group 1


## AgeGroup variable grouping

proc sql;  
&nbsp;&nbsp;&nbsp;&nbsp;  select agegroup, mean(age)  
&nbsp;&nbsp;&nbsp;&nbsp; group by agegroup    
&nbsp;&nbsp;&nbsp;&nbsp;  from df_dm3_agegroup;   
quit;

In [11]:
df_dm3_agegroup.groupby('AGEGROUP')['AGE'].mean()

AGEGROUP
age group 1    76.413502
age group 2    56.588235
Name: AGE, dtype: float64

In [12]:
df_dm3_agegroup.groupby('AGEGROUP')['AGE'].sum()

AGEGROUP
age group 1    18110.0
age group 2      962.0
Name: AGE, dtype: float64

### Drop variables in dataframe

In [13]:
### Prepare demographic data
df_dm4 = df_dm3.drop(['RACEC'],1)  # drop sex variable
df_dm4.head()

,AGE,AGEU,SEXC,SITEID,STUDYID,SUBJID,SEX,DOMAIN,COUNTRY,USUBJID,ETHNIC,RACE,RFSTDTC,RFENDTC
0,63.0,YEARS,Female,701,CDISCPILOT01,1015,F,DM,USA,CDISCPILOT01-701-1015,HISPANIC OR LATINO,WHITE,2014-01-02,2014-07-02
1,64.0,YEARS,Male,701,CDISCPILOT01,1023,M,DM,USA,CDISCPILOT01-701-1023,HISPANIC OR LATINO,WHITE,2012-08-05,2012-09-01
2,71.0,YEARS,Male,701,CDISCPILOT01,1028,M,DM,USA,CDISCPILOT01-701-1028,NOT HISPANIC OR LATINO,WHITE,2013-07-19,2014-01-14
3,74.0,YEARS,Male,701,CDISCPILOT01,1033,M,DM,USA,CDISCPILOT01-701-1033,NOT HISPANIC OR LATINO,WHITE,2014-03-18,2014-03-31
4,77.0,YEARS,Female,701,CDISCPILOT01,1034,F,DM,USA,CDISCPILOT01-701-1034,NOT HISPANIC OR LATINO,WHITE,2014-07-01,2014-12-30


## Convert object columns to datetime columns

In [14]:
for index2, _df2 in df_dm4.iterrows():
    # pd.Timestamp is the string function, so this needs to be done in for loop as Series, not as dataframe
    df_dm4.loc[index2, 'RFDTC_TS'] = pd.Timestamp(_df2.RFSTDTC)  # datetime creation  
    df_dm4.loc[index2, 'RFDTC_DT'] = pd.to_datetime(_df2.RFSTDTC)  # datetime creation

df_dm4['RF_YEAR'] = df_dm4.RFDTC_TS.dt.year #integer - year from RFSTDTC 
df_dm4['RF_MM'] = df_dm4.RFDTC_TS.dt.month  #integer - month from RFSTDTC
print(df_dm4.info())
df_dm4.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 254 entries, 0 to 253
Data columns (total 18 columns):
AGE         254 non-null float64
AGEU        254 non-null object
SEXC        254 non-null object
SITEID      254 non-null object
STUDYID     254 non-null object
SUBJID      254 non-null object
SEX         254 non-null object
DOMAIN      254 non-null object
COUNTRY     254 non-null object
USUBJID     254 non-null object
ETHNIC      254 non-null object
RACE        254 non-null object
RFSTDTC     254 non-null object
RFENDTC     254 non-null object
RFDTC_TS    254 non-null datetime64[ns]
RFDTC_DT    254 non-null datetime64[ns]
RF_YEAR     254 non-null int64
RF_MM       254 non-null int64
dtypes: datetime64[ns](2), float64(1), int64(2), object(13)
memory usage: 47.7+ KB
None


,AGE,AGEU,SEXC,SITEID,STUDYID,SUBJID,SEX,DOMAIN,COUNTRY,USUBJID,ETHNIC,RACE,RFSTDTC,RFENDTC,RFDTC_TS,RFDTC_DT,RF_YEAR,RF_MM
0,63.0,YEARS,Female,701,CDISCPILOT01,1015,F,DM,USA,CDISCPILOT01-701-1015,HISPANIC OR LATINO,WHITE,2014-01-02,2014-07-02,2014-01-02,2014-01-02,2014,1
1,64.0,YEARS,Male,701,CDISCPILOT01,1023,M,DM,USA,CDISCPILOT01-701-1023,HISPANIC OR LATINO,WHITE,2012-08-05,2012-09-01,2012-08-05,2012-08-05,2012,8
2,71.0,YEARS,Male,701,CDISCPILOT01,1028,M,DM,USA,CDISCPILOT01-701-1028,NOT HISPANIC OR LATINO,WHITE,2013-07-19,2014-01-14,2013-07-19,2013-07-19,2013,7
3,74.0,YEARS,Male,701,CDISCPILOT01,1033,M,DM,USA,CDISCPILOT01-701-1033,NOT HISPANIC OR LATINO,WHITE,2014-03-18,2014-03-31,2014-03-18,2014-03-18,2014,3
4,77.0,YEARS,Female,701,CDISCPILOT01,1034,F,DM,USA,CDISCPILOT01-701-1034,NOT HISPANIC OR LATINO,WHITE,2014-07-01,2014-12-30,2014-07-01,2014-07-01,2014,7


## String Operation

In [15]:
df_dm5 = df_dm4.copy()

### Lower Case Operation
df_dm5['RACE_L'] = df_dm5.RACE.str.lower()
df_dm5[['RACE', 'RACE_L']].head()

,RACE,RACE_L
0,WHITE,white
1,WHITE,white
2,WHITE,white
3,WHITE,white
4,WHITE,white


### Substring 
RACE_SUB = substr(RACE, 1,1)

In [16]:
### Substring string values
df_dm5['RACE_SUB'] = df_dm5.RACE.str[0:1]  # 
df_dm5.head()

,AGE,AGEU,SEXC,SITEID,STUDYID,SUBJID,SEX,DOMAIN,COUNTRY,USUBJID,ETHNIC,RACE,RFSTDTC,RFENDTC,RFDTC_TS,RFDTC_DT,RF_YEAR,RF_MM,RACE_L,RACE_SUB
0,63.0,YEARS,Female,701,CDISCPILOT01,1015,F,DM,USA,CDISCPILOT01-701-1015,HISPANIC OR LATINO,WHITE,2014-01-02,2014-07-02,2014-01-02,2014-01-02,2014,1,white,W
1,64.0,YEARS,Male,701,CDISCPILOT01,1023,M,DM,USA,CDISCPILOT01-701-1023,HISPANIC OR LATINO,WHITE,2012-08-05,2012-09-01,2012-08-05,2012-08-05,2012,8,white,W
2,71.0,YEARS,Male,701,CDISCPILOT01,1028,M,DM,USA,CDISCPILOT01-701-1028,NOT HISPANIC OR LATINO,WHITE,2013-07-19,2014-01-14,2013-07-19,2013-07-19,2013,7,white,W
3,74.0,YEARS,Male,701,CDISCPILOT01,1033,M,DM,USA,CDISCPILOT01-701-1033,NOT HISPANIC OR LATINO,WHITE,2014-03-18,2014-03-31,2014-03-18,2014-03-18,2014,3,white,W
4,77.0,YEARS,Female,701,CDISCPILOT01,1034,F,DM,USA,CDISCPILOT01-701-1034,NOT HISPANIC OR LATINO,WHITE,2014-07-01,2014-12-30,2014-07-01,2014-07-01,2014,7,white,W


### Find values in string
NUM_SEX = index(SEX, 'ale');

In [17]:
df_dm5['NUM_SEX'] = df_dm5.SEXC.str.find('ale')
df_dm5[['NUM_SEX', 'SEXC']].head()

,NUM_SEX,SEXC
0,3,Female
1,1,Male
2,1,Male
3,1,Male
4,3,Female


### numeric calcuation of two columns
NUM_AGE = AGE / NUM_SEX;

In [18]:
df_dm5['NUM_AGE'] = df_dm5.AGE / df_dm5.NUM_SEX
df_dm5[['NUM_AGE','AGE','NUM_SEX']].head()

,NUM_AGE,AGE,NUM_SEX
0,21.000000,63.0,3
1,64.000000,64.0,1
2,71.000000,71.0,1
3,74.000000,74.0,1
4,25.666667,77.0,3


## Write SAS datasets to the local drive

In [19]:
with open('./data/out/dm.xpt', 'wb') as f:
    xport.from_dataframe(df_dm5, f)## write DM 